In [1]:
## Search engine using Langchain

In [1]:
## Arxiv-Research
## Tool creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper # to run wikipediaQueryRun we need wrapper


In [2]:
# Here we have used inbuilt tool of wikipedia (following funct will have to conduct searches and page summaries)
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
# To run the wrapper we need queryrun
wiki = WikipediaQueryRun(api_wrapper = api_wrapper_wiki)

wiki.name

'wikipedia'

In [3]:
## Now creating the ArxivWraper

api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper = api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
## Combining above two tools using list
tools = [wiki, arxiv]


In [5]:
## To run the tools we need llm model
#  

In [6]:
## In the above cell we have create inbuit tools but now we will create custom tool

## Custom tools [RAG tools]

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
loader= WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

APIConnectionError: Connection error.

In [ ]:
## Converting retriever into tools
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any info about langchain")
retriever_tool.name

In [ ]:
tools = [wiki, arxiv, retriever_tool]

In [8]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\GenerativeAI\\Projects\\Search_Engine\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

In [ ]:
## Run all this tool with Agents and LLM Models
## Now we will run these tools using agents and llm model


## to combine Tools, LLM --> agentexecuter

from dotenv import load_dotenv
import openai
from langchain_groq import ChatGroq
load_dotenv()
import os 


groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENNAI_API_KEY")

llm = ChatGroq(groq_api_key = groq_api_key, model_name= "Llama3-8b-8192")


In [ ]:
## Prompt Template (you can use chatprompt template also, here from hub we are pulling 1 template)

from langchain import hub
prompt = hub.pull(" hwchase1/openai-functions-agent")
prompt.messages

In [ ]:
## to execute all together we will need to use chains

from langchain.agents import create_openai_tools_agent
agents = create_openai_tools_agent(llm, tools, prompt)
agents

In [ ]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent=agents, tools = tools, verbose=True)
agent_executer 

In [ ]:
agent_executer.invoke({"input": "Tell me about langsmith"})

In [ ]:
agent_executer.invoke({"input": "What is what is machine learning!"})